<a href="https://colab.research.google.com/github/roitraining/SparkforDataEngineers/blob/Development/Ch06_ClassificationAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import sys

rootpath = '/home/student/ROI/Spark/'
datapath = f'{rootpath}datasets/'
sys.path.append(rootpath)
import pyspark_helpers as pyh
from pyspark_helpers import *
sc, spark, conf = initspark()

import pandas as pd
import matplotlib as mp
import numpy
from matplotlib import pyplot as plt

from pyspark_helpers import display

initializing pyspark
pyspark initialized


### Let's read in a bank data set to try to predict if a potential borrower will default on their loan before lending to them

In [2]:
filename = 'bank.csv'
df = spark.read.csv(f'{datapath}/finance/{filename}', header = True, inferSchema = True)
display(df)

# Save a pointer to the raw data
dfRawFile = df


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,deposit
0,59,admin.,married,secondary,no,2343,yes,no,unknown,5,may,1042,1,-1,0,unknown,yes
1,56,admin.,married,secondary,no,45,no,no,unknown,5,may,1467,1,-1,0,unknown,yes
2,41,technician,married,secondary,no,1270,yes,no,unknown,5,may,1389,1,-1,0,unknown,yes
3,55,services,married,secondary,no,2476,yes,no,unknown,5,may,579,1,-1,0,unknown,yes
4,54,admin.,married,tertiary,no,184,no,no,unknown,5,may,673,2,-1,0,unknown,yes
5,42,management,single,tertiary,no,0,yes,yes,unknown,5,may,562,2,-1,0,unknown,yes
6,56,management,married,tertiary,no,830,yes,yes,unknown,6,may,1201,1,-1,0,unknown,yes
7,60,retired,divorced,secondary,no,545,yes,no,unknown,6,may,1030,1,-1,0,unknown,yes
8,37,technician,married,secondary,no,1,yes,no,unknown,6,may,608,1,-1,0,unknown,yes
9,28,services,single,secondary,no,5090,yes,no,unknown,6,may,1297,3,-1,0,unknown,yes


### Clean up the dataset by identifying the numeric and categorical features and target

In [3]:
# Let's just keep a few fields to start with for simplicity
import imp
imp.reload(pyh)

num = pyh.auto_numeric_features(df)
print (num)
cat = pyh.auto_categorical_features(df, suffix = None)
#cat = pyh.auto_categorical_features(df, suffix = ('ing', 'ion'))
print (cat)

numeric_features = ['age','balance', 'duration', 'pdays']
categorical_features = ['job', 'marital', 'education', 'housing', 'loan', 'contact', 'campaign', 'poutcome', 'deposit']

# numeric_features = ['balance', 'duration', 'age']
# categorical_features = ['marital', 'education']
target_label = 'default'


df = dfRawFile.select(numeric_features + categorical_features + [target_label])
display(df)
print(df.take(1))

['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
['job', 'marital', 'education', 'default', 'housing', 'loan', 'contact', 'month', 'poutcome', 'deposit']


,age,balance,duration,pdays,job,marital,education,housing,loan,contact,campaign,poutcome,deposit,default
0,59,2343,1042,-1,admin.,married,secondary,yes,no,unknown,1,unknown,yes,no
1,56,45,1467,-1,admin.,married,secondary,no,no,unknown,1,unknown,yes,no
2,41,1270,1389,-1,technician,married,secondary,yes,no,unknown,1,unknown,yes,no
3,55,2476,579,-1,services,married,secondary,yes,no,unknown,1,unknown,yes,no
4,54,184,673,-1,admin.,married,tertiary,no,no,unknown,2,unknown,yes,no
5,42,0,562,-1,management,single,tertiary,yes,yes,unknown,2,unknown,yes,no
6,56,830,1201,-1,management,married,tertiary,yes,yes,unknown,1,unknown,yes,no
7,60,545,1030,-1,retired,divorced,secondary,yes,no,unknown,1,unknown,yes,no
8,37,1,608,-1,technician,married,secondary,yes,no,unknown,1,unknown,yes,no
9,28,5090,1297,-1,services,single,secondary,yes,no,unknown,3,unknown,yes,no


[Row(age=59, balance=2343, duration=1042, pdays=-1, job='admin.', marital='married', education='secondary', housing='yes', loan='no', contact='unknown', campaign=1, poutcome='unknown', deposit='yes', default='no')]


### Explore numeric features. To see if there is any correlation between values

In [ ]:
%matplotlib inline

display(pyh.describe_numeric_features(df, numeric_features))
pyh.scatter_matrix(df, numeric_features)


### Use the helper function to reshape it for ML training

In [25]:
%matplotlib inline
# import imp
# imp.reload(pyh)

dfML, keydict = pyh.MakeMLDataFrame(df, categorical_features, numeric_features, target_label = 'default', target_is_categorical=True, return_key_dict = True)
display(dfML)
dfML.printSchema()
labelCnt = dfML.groupBy('label').count()
display(labelCnt)
print('keydict ' , keydict)
print(dfML.take(1))
print(df.take(1))

--->  ['age', 'balance', 'duration', 'pdays', 'default_Index', 'job_Vector', 'marital_Vector', 'education_Vector', 'housing_Vector', 'loan_Vector', 'contact_Vector', 'campaign_Vector', 'poutcome_Vector', 'deposit_Vector'] [Row(age=59, balance=2343, duration=1042, pdays=-1, default_Index=0.0, job_Vector=SparseVector(11, {3: 1.0}), marital_Vector=SparseVector(2, {0: 1.0}), education_Vector=SparseVector(3, {0: 1.0}), housing_Vector=SparseVector(1, {}), loan_Vector=SparseVector(1, {0: 1.0}), contact_Vector=SparseVector(2, {1: 1.0}), campaign_Vector=SparseVector(35, {0: 1.0}), poutcome_Vector=SparseVector(3, {0: 1.0}), deposit_Vector=SparseVector(1, {}))]


,label,features
0,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,0.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."
4,0.0,"(0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,0.0,"(1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,0.0,"(0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
8,0.0,"(0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,0.0,"(0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, ..."


root
 |-- label: double (nullable = false)
 |-- features: vector (nullable = true)



,label,count
0,0.0,10994
1,1.0,168


keydict  {'job': {'unemployed': 8.0, 'entrepreneur': 9.0, 'services': 4.0, 'retired': 5.0, 'management': 0.0, 'unknown': 11.0, 'self-employed': 6.0, 'student': 7.0, 'blue-collar': 1.0, 'admin.': 3.0, 'housemaid': 10.0, 'technician': 2.0}, 'marital': {'married': 0.0, 'divorced': 2.0, 'single': 1.0}, 'education': {'secondary': 0.0, 'unknown': 3.0, 'tertiary': 1.0, 'primary': 2.0}, 'housing': {'no': 0.0, 'yes': 1.0}, 'loan': {'no': 0.0, 'yes': 1.0}, 'contact': {'telephone': 2.0, 'cellular': 0.0, 'unknown': 1.0}, 'campaign': {2: 1.0, 4: 3.0, 23: 24.0, 43: 27.0, 33: 30.0, 25: 26.0, 15: 15.0, 1: 0.0, 8: 7.0, 30: 23.0, 26: 25.0, 5: 4.0, 29: 29.0, 21: 17.0, 28: 35.0, 6: 5.0, 16: 16.0, 20: 21.0, 11: 10.0, 3: 2.0, 13: 11.0, 9: 8.0, 17: 14.0, 18: 18.0, 14: 13.0, 41: 33.0, 22: 22.0, 63: 34.0, 31: 31.0, 27: 32.0, 12: 12.0, 7: 6.0, 24: 20.0, 32: 28.0, 10: 9.0, 19: 19.0}, 'poutcome': {'other': 3.0, 'failure': 1.0, 'unknown': 0.0, 'success': 2.0}, 'deposit': {'no': 0.0, 'yes': 1.0}, 'default': {'no': 

In [ ]:
labelCnt.toPandas().plot(kind = 'bar')


### Save the Vectorized file in case we want to use it again 

In [ ]:
# dfML.write.format('parquet').mode('overwrite').save('testsave')
# dfML0 = spark.read.format('parquet').load('testsave')
# dfML0.printSchema()
# display(dfML0)

### Split it into training and testing sets

In [12]:
#dfML = dfML0
train, test = dfML.randomSplit([.7,.3], seed = 100)
print (f'Training set row count {train.count()}')
print (f'Testing set row count {test.count()}')
display(train.groupBy('label').count())
display(test.groupBy('label').count())
#print(test.take(1))


Training set row count 7864
Testing set row count 3298


,label,count
0,0.0,7752
1,1.0,112


,label,count
0,0.0,3242
1,1.0,56


[Row(label=0.0, features=SparseVector(63, {0: 1.0, 11: 1.0, 13: 1.0, 16: 1.0, 17: 1.0, 18: 1.0, 20: 1.0, 55: 1.0, 58: 1.0, 59: 67.0, 60: 1287.0, 61: 511.0, 62: -1.0}))]


### Import the Decision Tree classifier and train it

In [6]:
from pyspark.ml.classification import DecisionTreeClassifier
dt = DecisionTreeClassifier(featuresCol = 'features', labelCol = 'label', maxDepth = 3)
dtModel = dt.fit(train)
print('DT Trained')

filename1 = filename.replace('.','_') + '_DT_trainedModel'
dtModel.write().overwrite().save(filename1)
print('DT Saved')


DT Trained
DT Saved


### Normally there are a lot of steps to predict and test. We have built a helper function to bundle all that up.
Take a look at the source code for it to see those indivual steps.

In [7]:
dtPredictions, dtLog = pyh.predict_and_evaluate(dtModel, test)


Test Area Under ROC 0.8520368819952411
+-----+-------------+----------+--------------------+
|label|rawPrediction|prediction|         probability|
+-----+-------------+----------+--------------------+
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911789652...|
|  0.0|[5870.0,25.0]|       0.0|[0.99575911

### Now let's try Logistic Regression

In [ ]:
from pyspark.ml.classification import LogisticRegression
lr = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10)
lrModel = lr.fit(train)
print('LR Trained')

filename1 = filename.replace('.','_') + '_LR_trainedModel'
lrModel.write().overwrite().save(filename1)
print('LR Saved')


### Normally you should be able to load the trained model, but for some reason it's not working correctly on this VM

In [ ]:
#lrModel2 = LogisticRegression.load(filename1)

### See the test results as before, but LR has some extra options

In [ ]:
lrPredictions, lrLog = pyh.predict_and_evaluate(lrModel, test, showModel = False)

### Let's try different thresholds to see if we can tweak the false positive/negative balance or improve the overall accuracy

In [ ]:
lr2 = LogisticRegression(featuresCol = 'features', labelCol = 'label', maxIter=10, threshold = .1).fit(train)

lr2Predictions, lr2Log = pyh.predict_and_evaluate(lr2, test, showModel = False)

### After a while it's the same thing over and over, but try out as many models as possible to see which works best


In [ ]:
from pyspark.ml.classification import RandomForestClassifier
rf = RandomForestClassifier(featuresCol = 'features', labelCol = 'label')
rfModel = rf.fit(train)
print('RF Trained')

filename1 = filename.replace('.','_') + '_RF_trainedModel'
rfModel.write().overwrite().save(filename1)
print('RF Saved')



In [ ]:
rfPredictions, rfLog = pyh.predict_and_evaluate(rfModel, test)



### Try Gradient Boost

In [ ]:
from pyspark.ml.classification import GBTClassifier
gbt = GBTClassifier(maxIter=10)
gbtModel = gbt.fit(train)
print ('GBT Trained')

filename1 = filename.replace('.','_') + '_GBT_trainedModel'
rfModel.write().overwrite().save(filename1)
print ('GBT Saved')



In [ ]:
gbtPredictions, gbtLog = pyh.predict_and_evaluate(gbtModel, test)



### Try Neural Networks

In [ ]:
from pyspark.ml.classification import MultilayerPerceptronClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# specify layers for the neural network:
# input layer of size 13 (features), two intermediate of size 5 and 4
# and output of size 2 (classes)
layers = [13, 5, 4, 2]

nn = MultilayerPerceptronClassifier(maxIter=100, layers=layers, blockSize=128, seed=1234)
nnModel = nn.fit(train)
print ('NN Trained')

filename1 = filename.replace('.','_') + '_NN_trainedModel'
nnModel.write().overwrite().save(filename1)
print ('NN Saved')



In [ ]:
nnPredictions = nnModel.transform(test)
print(type(nnPredictions))
print(nnPredictions.take(1))
nnPredictions.printSchema()
print (nnPredictions.count())

#nnPredictions, nnLog = pyh.predict_and_evaluate(nnModel, test)
##nnPredictions.take(1)
# predictionAndLabels = nnPredictions.select("prediction", "label")
# #display(predictionAndLabels)
# print(predictionAndLabels.collect())
# evaluator = MulticlassClassificationEvaluator(metricName="accuracy")
# print("Test set accuracy = " + str(evaluator.evaluate(predictionAndLabels)))

### Work in progress


In [103]:
d = {'job':'unemployed', 'marital':'divorced', 'amount':100, 'sum':200}
#{'job': {'unemployed': 8.0, 'entrepreneur': 9.0, 'services': 4.0, 'retired': 5.0, 'management': 0.0, 'unknown': 11.0, 'self-employed': 6.0, 'student': 7.0, 'blue-collar': 1.0, 'admin.': 3.0, 'housemaid': 10.0, 'technician': 2.0}, 'marital': {'married': 0.0, 'divorced': 2.0, 'single': 1.0}, 'education': {'secondary': 0.0, 'unknown': 3.0, 'tertiary': 1.0, 'primary': 2.0}
import imp
imp.reload(pyh)

def makeSparseVector(index, count):
    from  pyspark.mllib.linalg import SparseVector

    if index == count - 1:
        return SparseVector(count - 1, [], [])
    return SparseVector(count - 1, [index], [1.0])

def predictOne(model, keydict, **kwargs):
    
    from pyspark.ml.feature import VectorAssembler
    x = kwargs.copy()

    categorical_features = []
    for key in keydict:
        if key in kwargs.keys():
            x[key] = keydict[key].get(kwargs[key])
            #print ('*', x, key, x[key])
            categorical_features.append(key)
            print ('***', key, x[key], keydict[key], keydict[key].keys())
            x[key] = makeSparseVector(x[key], len(keydict[key].keys()))
            
    df = spark.createDataFrame(sc.parallelize([x]))
    print(df.take(1))
    numeric_features = list(set(kwargs.keys()).difference(set(categorical_features)))
    print(categorical_features, numeric_features)
    
    assemblerInputs = numeric_features + categorical_features 
    assembler = VectorAssembler(inputCols=assemblerInputs, outputCol="features")
    df = assembler.transform(df).drop(*(numeric_features)) # + categorical_features))
    df.printSchema()
    print('--->', df.take(1))
    
#     for col in categorical_features:
#         df = df.withColumnRenamed(col, col + '_Vector')
#     df = AssembleFeatures(df, categorical_features, numeric_features).select('features')
#     df.show()
    #z = pyh.MakeMLDataFrame(df, categorical_columns, numeric_columns, None, target_is_categorical = False, return_key_dict = False, skip_string_indexer = True)
    #return z
    #print(z.take(1))
    #predictions = model.transform(z)
    #return predictions

    
#help(SparseVector)
# x = SparseVector(1, [1,0])
# print (x)
#from pyspark.mllib.
#print(SparseVector)

predictOne(None, keydict, age = 30, marital = 'single', job = 'admin.', duration = 1042)

# predict = [dict(age=30,marital='single'), dict(age=59,marital='married'), dict(age=70,marital='divorced'), dict(age=90,marital='widow')]
# predict = spark.createDataFrame(sc.parallelize(predict))
# predict.show()
# x, x0 = pyh.StringIndexEncode(predict, ['marital'], return_key_dict = True)
# x.show()
# print (x0)
# y = pyh.OneHotEncode(x, ['marital'])
# y.show()

# # predict = dict(age=59, balance=2343, duration=1042, pdays=-1, job='admin.', marital='married', education='secondary', housing='yes', loan='no', contact='unknown', campaign=1, poutcome='unknown', deposit='yes')
# # print(predict)
# # predict = spark.createDataFrame(sc.parallelize([predict]))
# # print(predict)
# # x = dtModel.transform(predict)

# # x = predictOne(None, keydict, **d)
# # x.printSchema()
# # print(x.columns)
# # print(type(x.features))
# # help(x)
# # x = predictOne(dtModel, keydict, age=59, balance=2343, duration=1042, pdays=-1, job='admin.', marital='married', education='secondary', housing='yes', loan='no', contact='unknown', campaign=1, poutcome='unknown', deposit='yes')
# # print(x)
# # print(x.count())
# # x.show()



# #[Row(label=0.0, features=SparseVector(63, {3: 1.0, 11: 1.0, 13: 1.0, 17: 1.0, 19: 1.0, 20: 1.0, 55: 1.0, 59: 59.0, 60: 2343.0, 61: 1042.0, 62: -1.0}))]
# #[Row(age=59, balance=2343, duration=1042, pdays=-1, job='admin.', marital='married', education='secondary', housing='yes', loan='no', contact='unknown', campaign=1, poutcome='unknown', deposit='yes', default='no')]
# print (y.take(1))

# import numpy as np
# a = np.array([1,0])
# print (a)
# b = y.take(1)[0]['marital_Vector']
# print (b)
# a = SparseVector(2,[0.0],[1.0]) 
# print(a)
# print(type(a), type(b))
# print (a == b)
# #help(a)

# def makeSparseVector(index, count):
#     if index == count - 1:
#         return SparseVector(count - 1, [], [])
#     return SparseVector(count - 1, [index], [1.0])

# print (makeSparseVector(0, 4))
# print (makeSparseVector(1, 4))
# print (makeSparseVector(2, 4))
# print (makeSparseVector(3, 4))

*** job 3.0 {'unemployed': 8.0, 'entrepreneur': 9.0, 'services': 4.0, 'retired': 5.0, 'management': 0.0, 'unknown': 11.0, 'self-employed': 6.0, 'student': 7.0, 'blue-collar': 1.0, 'admin.': 3.0, 'housemaid': 10.0, 'technician': 2.0} dict_keys(['unemployed', 'entrepreneur', 'services', 'retired', 'management', 'unknown', 'self-employed', 'student', 'blue-collar', 'admin.', 'housemaid', 'technician'])
*** marital 1.0 {'married': 0.0, 'divorced': 2.0, 'single': 1.0} dict_keys(['married', 'divorced', 'single'])
[Row(age=30, duration=1042, job=SparseVector(11, {3: 1.0}), marital=SparseVector(2, {1: 1.0}))]
['job', 'marital'] ['duration', 'age']


IllegalArgumentException: 'Data type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> of column job is not supported.\nData type struct<type:tinyint,size:int,indices:array<int>,values:array<double>> of column marital is not supported.'